<a href="https://colab.research.google.com/github/NehaKumari500092077/Machine-Learning-Lab/blob/main/Session_6_Assignments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Assignment 6**: Predicting Bike Sharing Demand with Neural Networks





In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor
import matplotlib.pyplot as plt

# 1. Load and preprocess the [Bike Sharing dataset](https://archive.ics.uci.edu/dataset/275/bike+sharing+dataset) **[1 Marks]**

*   Check for missing and duplicate values
*   Remove it (If found)

In [3]:
dataset = pd.read_csv('https://raw.githubusercontent.com/NehaKumari500092077/Machine-Learning-Lab/main/hour.csv')
dataset.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [4]:
#drop instant, dteday, casual, and registered column
dataset = dataset.drop(['instant', 'dteday', 'casual', 'registered'], axis=1)
dataset.head()


,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,16
1,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,40
2,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,32
3,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,13
4,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,1


In [5]:
# Check Duplicates
duplicate_values = dataset.duplicated().sum()
print("No. of duplicate values: ",duplicate_values)

# Remove duplicate values
dataset = dataset.drop_duplicates()
print("No. of duplicate values: ",dataset.duplicated().sum())

# Check for missing values
missing_values = dataset.isnull().sum()
print("No. of missing values: ",missing_values)
# As there is no missing values present in the dataset so no need to remove it

No. of duplicate values:  2
No. of duplicate values:  0
No. of missing values:  season        0
yr            0
mnth          0
hr            0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
cnt           0
dtype: int64


# 2. Data Pre-processing **[1 Marks]**

*   Perform one-hot encoding of the categorical features when necessary (not necessary if there is a natural ordering between categories)
*   Perform feature scaling



In [6]:
categorical_features = ['season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit']
continuous_features = ['temp', 'atemp', 'hum', 'windspeed']

# Perform one-hot encoding on categorical features
encoded_dataset = pd.get_dummies(dataset, columns=categorical_features, drop_first=True)

# Perform feature scaling on continuous features
scaler = StandardScaler()
encoded_dataset[continuous_features] = scaler.fit_transform(dataset[continuous_features])

# 3. Data Splitting **[1  Marks]**

*   Divide the data into training, validation and test sets (70%-15%-15%).



In [7]:
# Seperate features and target
X = encoded_dataset.drop('cnt', axis=1)
Y = encoded_dataset['cnt']

# Split data into training, validation and test sets
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.3, random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42)


# 4. Model Design: Design a neural network having N hidden layers **[7  Marks]**  

*   The number of units in each hidden layer as 128, 64, 32, and so on
*   Activation function in each hidden layer: ReLU
*   What activation function should you choose for the output layer for this prediction problem?



In [11]:
learning_rate = 0.01
epochs = 100
#'hidden_layer_sizes': [(128,), (128, 64), (128, 64, 32), (128, 64, 32, 16), (128, 64, 32, 16, 8), (128, 64, 32, 16, 8, 4), (128, 64, 32, 16, 8, 4, 2)]
param_grid = {
    'hidden_layer_sizes': [(128,), (128, 64), (128, 64, 32), (128, 64, 32, 16)]
}
mlp_regressor_model = MLPRegressor(
    activation='relu',
    solver='sgd',
    learning_rate_init=learning_rate,
    max_iter=epochs,
    random_state=42,
    batch_size=X_train.shape[0]
    )

# 5. Cost function: Define a cost function suitable for this problem **[3  Marks]**

*   Use the SGD optimizer to optimize this cost function using backpropagation.



# 6. Perform cross-validation to find the best value of N using grid search. **[2  Marks]**



In [12]:
# Perform cross-validation to find the best value of N using grid search
gridSearch = GridSearchCV(mlp_regressor_model, param_grid, cv=5, scoring='neg_mean_squared_error')
gridSearch.fit(X_train, Y_train)

# Get the best value of N
best_value_of_N = gridSearch.best_params_['hidden_layer_sizes']

# Train the model on best value of N
best_mlp_regressor_model = MLPRegressor(
    hidden_layer_sizes= best_value_of_N,
    activation='relu',
    solver='sgd',
    learning_rate_init=learning_rate,
    max_iter=epochs,
    random_state=42,
    batch_size=X_train.shape[0]
    )

best_mlp_regressor_model.fit(X_train, Y_train)


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:609: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_base.py:173: RuntimeWarning: overflow encountered in square
  return ((y_true - y_pred) ** 2).mean() / 2
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:181: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:192: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:192: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the opt

ValueError: Solver produced non-finite parameter weights. The input data may contain large values and need to be preprocessed.

# 7. Loss Plots: **[1 Marks]**

*  Plot the training and validation losses for each value of N (on the same
graph).



In [ ]:
# Get the training and validation losses
trainingLosses = best_mlp_regressor_model.loss_curve_
validationLosses = []

for layer_size in param_grid['hidden_layer_sizes']:
  model = MLPRegressor(
    hidden_layer_sizes= layer_size,
    activation='relu',
    solver='sgd',
    learning_rate_init=learning_rate,
    max_iter=epochs,
    random_state=42,
    batch_size=X_train.shape[0]
    )
  model.fit(X_train, Y_train)
  y_pred = model.predict(X_val)
  validation_loss = mean_squared_error(Y_val, y_pred)
  validationLosses.append(validation_loss)


# Plot the training and validation losses for each value of N
plt.plot(trainingLosses, label='Training Loss')
plt.plot(validationLosses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Losses for Different Number of Hidden Layers')
plt.legend()
plt.show()

# 8. What is the *best value of the number of hidden layers (N)* for this problem? Justify your answer. **[2  Marks]**



# 9. For the model trained with the best value of N that you reported in the previous question, *report the MSE on the test set*.  **[1  Marks]**



In [ ]:
model_predicted_value = best_mlp_regressor_model.predict(X_test)
mse = mean_squared_error(Y_test, model_predicted_value)
print("Mean Squared Error (MSE) on the test set: ", mse)


# 10. Show a *scatter plot of the predictions against the true values* for the predictions for the best model that you obtained. **[1  Marks]**

In [ ]:
# scatter plot of the predictions against the true values for the predictions for the best model
plt.scatter(Y_test, model_predicted_value)
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.title('Scatter Plot of Predictions vs True Values')
plt.show()
